In [1]:
# Romain Puech, 2024

## uncomment below if some libraries are missing:
# pip install -r requirements.txt

import numpy as np
import time
from displays import display_grid, save_grid_image, create_video_from_images
from simulations import propagate_wildfire, propagate_smoke
from Drone import Drone

In [2]:
### Simulation initialization and loop

###### Initialize simulation parameters
N = 100  # Grid size
T = 100  # Number of time steps

initial_wildfire_cells = [(10, 3)]  # Initial fire location(s)
p = -3.5  # Base fire spread logodds
burn_time = 10  # Number of steps before a burning cell becomes burnt

wind_speed = 5  # Wind strength
wind_direction = "right"  # Wind direction

diffusion_percentage = 0.4 # Percentage of smoke that diffuses to neighboring cells
n_drones = 5
#####

###### Initialize grids
grid = np.zeros((N, N), dtype=int)
for x, y in initial_wildfire_cells:
    grid[x, y] = 1
burn_counters = np.zeros((N, N), dtype=int) # for how long the cell has been burning
smoke_grid = np.zeros((N, N), dtype=float)

drones = [Drone(*np.random.randint(0,N,2)) for _ in range(n_drones)]

print("Initial Grid:")
display_grid(grid,smoke_grid,drones,{'fire','smoke','drones'})
save_grid_image(grid, smoke_grid, drones, {'smoke','fire','drones'}, 0)
#####

####### Simulation loop
start = time.time()
drone_signals = []
for t in range(1,T+1):
    print(f"Time Step {t}")
    # simulate wildfire and smoke spread
    grid, burn_counters = propagate_wildfire(grid, burn_time, burn_counters, p, wind_speed, wind_direction)
    smoke_grid = propagate_smoke(grid, smoke_grid, wind_speed, wind_direction)

    # route drones
    time_step_signals = []
    for drone in drones:
        drone.route(grid, smoke_grid, wind_direction)
        time_step_signals.append(drone.fire_alert)
    drone_signals.append(time_step_signals)

    # save result
    save_grid_image(grid, smoke_grid, drones, {'smoke','fire','drones'}, t)

# save final result as a video
output_video = "wildfire_simulation.mp4"
create_video_from_images(frames_per_image=3)
end = time.time()
print(f"Simulation completed in {round(end-start)} seconds. Video '{output_video}' created.")

# Compute objective values
first_signal = 'UNDETECTED'
for i in range(len(drone_signals)):
    if any(drone_signals[i]):
        first_signal = i
        break

print(f"Fire detected after {first_signal} time steps")


Initial Grid:
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
........................................................................D....